In [ ]:

# Install required libraries (Run this if not already installed)
!pip install sqlalchemy pymysql

In [ ]:

from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base, sessionmaker

# Replace these credentials with your MySQL details
DATABASE_URL = "mysql+pymysql://<username>:<password>@<host>/<database>"

# Create the engine
engine = create_engine(DATABASE_URL, echo=True)

# Define the Base class for the ORM models
Base = declarative_base()

# Create a session maker
Session = sessionmaker(bind=engine)
session = Session()
    


# Database Table Structures

## Users Table
| Column Name | Data Type     | Constraints          |
|-------------|---------------|----------------------|
| id          | INT           | PRIMARY KEY          |
| name        | VARCHAR(50)   | NOT NULL             |
| email       | VARCHAR(100)  | UNIQUE, NOT NULL     |

---

## Posts Table
| Column Name | Data Type      | Constraints            |
|-------------|----------------|------------------------|
| id          | INT            | PRIMARY KEY            |
| title       | VARCHAR(255)   | NOT NULL               |
| content     | TEXT           |                        |
| user_id     | INT            | FOREIGN KEY (users.id) |

---

## Tags Table
| Column Name | Data Type     | Constraints        |
|-------------|---------------|--------------------|
| id          | INT           | PRIMARY KEY        |
| name        | VARCHAR(50)   | UNIQUE, NOT NULL   |

---

## Post_Tag Table
| Column Name | Data Type | Constraints            |
|-------------|-----------|------------------------|
| post_id     | INT       | PRIMARY KEY, FOREIGN KEY (posts.id) |
| tag_id      | INT       | PRIMARY KEY, FOREIGN KEY (tags.id)  |

```sql
CREATE TABLE users (
    id INT PRIMARY KEY,
    name VARCHAR(50) NOT NULL,
    email VARCHAR(100) UNIQUE NOT NULL
);

-- Assuming you have a posts table with a user_id foreign key
CREATE TABLE posts (
    id INT PRIMARY KEY,
    title VARCHAR(255) NOT NULL,
    content TEXT,
    user_id INT,
    FOREIGN KEY (user_id) REFERENCES users(id)
);

CREATE TABLE posts (
    id INT PRIMARY KEY,
    title VARCHAR(100) NOT NULL,
    content VARCHAR(100) NOT NULL,
    user_id INT NOT NULL,
    FOREIGN KEY (user_id) REFERENCES users(id)
);

-- Association table for the Many-to-Many relationship
CREATE TABLE post_tag (
    post_id INT,
    tag_id INT,
    PRIMARY KEY (post_id, tag_id),
    FOREIGN KEY (post_id) REFERENCES posts(id),
    FOREIGN KEY (tag_id) REFERENCES tags(id)
);
CREATE TABLE tags (
    id INT PRIMARY KEY,
    name VARCHAR(50) UNIQUE NOT NULL
);

```

In [ ]:

from sqlalchemy import Column, Integer, String, ForeignKey, MetaData 

# User model
class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(50), nullable=False)
    email = Column(String(100), unique=True, nullable=False)
    
    
    def __repr__(self):
        return f"<User(name={self.name}, email={self.email})>"

# Post model
class Post(Base):
    __tablename__ = 'posts'
    
    id = Column(Integer, primary_key=True)
    title = Column(String(100), nullable=False)
    content = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    
    
    def __repr__(self):
        return f"<Post(title={self.title}, user={self.user.name})>"



# Create all tables in the database
Base.metadata.create_all(engine)
    

In [ ]:
user1 = User(name="Alice", email="alice@example.com")
user2 = User(name="Bob", email="bob@example.com")

# Add users to the session
session.add(user1)
session.add(user2)

# Commit the session to save users
session.commit()

# Add posts for the users
post1 = Post(title="First Post", content="This is Alice's first post.", user_id=user1.id)
post2 = Post(title="Second Post", content="This is Bob's first post.", user_id=user2.id)

# Add posts to the session
session.add(post1)
session.add(post2)

# Commit the session to save posts
session.commit()

display("Data inserted successfully!")


In [ ]:
users = session.query(User).all()
display("Users:")
for user in users:
    display(user)

# Query all posts
posts = session.query(Post).all()
display("\nPosts:")
for post in posts:
    display(post)


In [ ]:

# Query for users with the specified name
users = session.query(User).filter(User.name == "Alice").all()
for user in users:
    display(user)

# Query for posts by the given user ID
posts = session.query(Post).filter(Post.user_id == 1).all()

for post in posts:
    display(post)


# Use filter_by for a simpler filter (equality only)
some_email="alice@example.com"
user = session.query(User).filter_by(email=some_email).first()
if user:
    display(f"User with email '{some_email}': {user}")
else:
    display(f"No user found with email '{some_email}'")
    

# Use a wildcard search with `like`
keyword="First"
posts = session.query(Post).filter(Post.title.like(f"%{keyword}%")).first()
display(f"Posts containing '{keyword}' in the title:")
for post in posts:
    display(post)

In [ ]:

# Filter users by multiple conditions (name and email)
from sqlalchemy import and_
users = session.query(User).filter(and_(User.name == "Alice", User.email == "alice@example.com")).all()
display("Users with name 'Alice' and email 'alice@example.com':")
for user in users:
    display(user)


# Filter users who match either condition
from sqlalchemy import or_
users = session.query(User).filter(or_(User.name == "Alice", User.email == "bob@example.com")).all()
display("Users with name 'Alice' or email 'bob@example.com':")
for user in users:
    display(user)


In [ ]:
user_to_delete = session.query(User).filter_by(email="alice@example.com").first()

if user_to_delete:
    # Delete the user
    session.delete(user_to_delete)
    session.commit()
    display(f"Deleted user: {user_to_delete.name}")

# Delete all posts
session.query(Post).delete()
session.commit()
display("Deleted all posts.")

In [ ]:
# Drop a specific table (User)
User.__table__.drop(engine)
display("Users table dropped successfully!")
# Drop all tables in the metadata
metadata.drop_all(engine)
display("All tables dropped successfully!")

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey 
from sqlalchemy.orm import relationship 

 

# User model
class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String(50), nullable=False)
    email = Column(String(100), unique=True, nullable=False)

    # One-to-Many relationship with Post
    posts = relationship("Post", back_populates="user")

    def __repr__(self):
        return f"<User(name={self.name}, email={self.email})>"

# Post model
class Post(Base):
    __tablename__ = 'posts'

    id = Column(Integer, primary_key=True)
    title = Column(String(100), nullable=False)
    content = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)

    # Many-to-Many relationship with Tag
    tags = relationship("Tag", secondary="post_tag", back_populates="posts")
    user = relationship("User", back_populates="posts")

    def __repr__(self):
        return f"<Post(title={self.title}, user={self.user.name})>"

# Tag model
class Tag(Base):
    __tablename__ = 'tags'

    id = Column(Integer, primary_key=True)
    name = Column(String(50), unique=True, nullable=False)

    # Many-to-Many relationship with Post
    posts = relationship("Post", secondary="post_tag", back_populates="tags")

    def __repr__(self):
        return f"<Tag(name={self.name})>"

# Association table for Many-to-Many relationship (still defined explicitly but referenced implicitly)
class PostTag(Base):
    __tablename__ = 'post_tag'
    post_id = Column(Integer, ForeignKey('posts.id'), primary_key=True)
    tag_id = Column(Integer, ForeignKey('tags.id'), primary_key=True)

# Create all tables in the database
Base.metadata.create_all(engine)

 


In [ ]:

# Create a user
user = User(name="Alice", email="alice@example.com")

# Create posts for the user
post1 = Post(title="First Post", content="Alice's first post", user=user)
post2 = Post(title="Second Post", content="Alice's second post", user=user)

# Create tags
tag1 = Tag(name="Introduction")
tag2 = Tag(name="SQLAlchemy")

# Associate tags with posts
post1.tags.extend([tag1, tag2])
post2.tags.append(tag2)

# Add and commit the data
session.add(user)
session.commit()

# Verify the data
display("Users:", session.query(User).all())
display("Posts:", session.query(Post).all())
display("Tags:", session.query(Tag).all())
    

In [ ]:

# Fetch all posts by a user
alice_posts = session.query(User).filter(User.name == "Alice").first().posts
display("Alice's Posts:", alice_posts)

# Fetch all tags for a specific post
post_tags = session.query(Post).filter(Post.title == "First Post").first().tags
display("Tags for 'First Post':", post_tags)

# Fetch all posts with a specific tag
tag_posts = session.query(Tag).filter(Tag.name == "SQLAlchemy").first().posts
display("Posts with tag 'SQLAlchemy':", tag_posts)
    

In [ ]:

# Update a post's title
post_to_update = session.query(Post).filter(Post.title == "First Post").first()
post_to_update.title = "Updated First Post"
session.commit()

# Remove a tag from a post
post_to_update.tags.remove(tag1)
session.commit()

# Delete a user and cascade to posts
user_to_delete = session.query(User).filter(User.name == "Alice").first()
session.delete(user_to_delete)
session.commit()

# Verify the changes
display("Remaining Users:", session.query(User).all())
display("Remaining Posts:", session.query(Post).all())
display("Remaining Tags:", session.query(Tag).all())
    